# Creating a feature matrix from a DB Query (Opto Sessions)

In [15]:
import sys
sys.path.append('/Users/Sabatini Lab/GitHub/mouse_bandit/data_preprocessing_code/opto_preprocessing')
sys.path.append('/Users/Sabatini Lab/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing_opto as bp
from sklearn import preprocessing
import sys
import os
%matplotlib inline 

## Retrieve names of sessions you want from the DB

In [16]:
#load in data base
db = pd.read_csv('/Users/Sabatini Lab/GitHub/mouse_bandit/mike_session_record.csv',index_col=0)

### Query all 90-10 sessions where performance > 0.7 block structure was 50 and had no laser trials

In [17]:
r = db[((db['Left Reward Prob'] == 0.9) |  (db['Right Reward Prob'] == 0.9))].copy()
r = r[r['p(high Port)'] > 0.5].copy()
r = r[r['Block Range Min'] == 50].copy()
r = r[r['Laser Stim Prob'] == 0.1].copy()

session_names = r['Session ID'].values 

In [18]:
r.tail(10)

,Owner,Session ID,Mouse ID,Date,Phase,Laser Stim Prob,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
1061,mike,07212017_SOM45,SOM45,2017-07-21 00:00:00,2,0.1,0.1,0.9,50,50,353,4,235,0.72,2,1.0,40,40
1072,mike,07252017_SOM44,SOM44,2017-07-25 00:00:00,2,0.1,0.1,0.9,50,50,84,0,49,0.62,2,1.0,40,40
1073,mike,07252017_SOM45,SOM45,2017-07-25 00:00:00,2,0.1,0.9,0.1,50,50,449,7,346,0.84,2,1.0,40,40
1090,mike,07282017_SOM44,SOM44,2017-07-28 00:00:00,2,0.1,0.9,0.1,50,50,257,3,181,0.75,2,1.0,40,40
1091,mike,07282017_SOM45,SOM45,2017-07-28 00:00:00,2,0.1,0.9,0.1,50,50,515,7,365,0.77,2,1.0,40,40
1097,mike,08012017_SOM41,SOM41,2017-08-01 00:00:00,2,0.1,0.1,0.9,50,50,545,8,432,0.87,2,1.0,40,40
1098,mike,08012017_SOM44,SOM44,2017-08-01 00:00:00,2,0.1,0.9,0.1,50,50,105,1,73,0.79,2,1.0,40,40
1105,mike,08032017_SOM41,SOM41,2017-08-03 00:00:00,2,0.1,0.1,0.9,50,50,289,4,230,0.91,2,1.0,40,40
1106,mike,08032017_SOM44,SOM44,2017-08-03 00:00:00,2,0.1,0.9,0.1,50,50,273,4,223,0.86,2,1.0,40,40
1107,mike,08032017_SOM45,SOM45,2017-08-03 00:00:00,2,0.1,0.1,0.9,50,50,536,8,423,0.87,2,1.0,40,40


In [19]:
r.shape

(22, 18)

## load in csv files (from running exportTrials.m) added in 'Laser Given' column in trials for new data format following opto changes

In [20]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given','Laser Given']

root_dir = '/Users/Sabatini Lab/GitHub/mouse_bandit/data/mike_data/trial_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

## convert into 1 feature matrix

In [21]:
for i,df in enumerate(trial_df):
    
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    

In [22]:
master_matrix.shape

(8153, 54)

In [23]:
master_matrix['Higher p port'].mean()

0.80375321967373969

In [24]:
master_matrix['Laser'].mean()

0.1006991291549123

## Stephenson et al acquired ~6500 trials per animal

In [25]:
master_matrix['Session ID'].unique()

array(['07122017_PV30', '07122017_SOM42', '07142017_SOM42',
       '07172017_SOM41', '07172017_SOM44', '07172017_SOM45',
       '07182017_SOM42', '07192017_SOM41', '07192017_SOM44',
       '07192017_SOM45', '07202017_SOM42', '07212017_SOM44',
       '07212017_SOM45', '07252017_SOM44', '07252017_SOM45',
       '07282017_SOM44', '07282017_SOM45', '08012017_SOM41',
       '08012017_SOM44', '08032017_SOM41', '08032017_SOM44',
       '08032017_SOM45'], dtype=object)

In [27]:
master_matrix.tail(10)

,Mouse ID,Session ID,Trial,Block Trial,Block Reward,Port Streak,Reward Streak,10_Port,10_Reward,10_ITI,...,1_Reward,1_ITI,1_trialDuration,0_ITI,0_trialDuration,Decision,Switch,Higher p port,Reward,Laser
516,SOM45,08032017_SOM45,527.0,30.0,20.0,8.0,8.0,0.406,0.0,0.0,...,0.0,1.0,2.985,0.563,18.320,1.0,1.0,0.0,0.0,0.0
517,SOM45,08032017_SOM45,528.0,31.0,20.0,1.0,-1.0,0.513,1.0,0.0,...,1.0,0.0,18.320,0.406,2.566,0.0,1.0,1.0,1.0,0.0
518,SOM45,08032017_SOM45,529.0,32.0,21.0,1.0,1.0,0.337,0.0,1.0,...,0.0,1.0,2.566,0.513,1.973,0.0,0.0,1.0,1.0,0.0
519,SOM45,08032017_SOM45,530.0,33.0,22.0,2.0,2.0,0.550,0.0,1.0,...,0.0,1.0,1.973,0.337,2.289,0.0,0.0,1.0,0.0,0.0
520,SOM45,08032017_SOM45,531.0,34.0,22.0,3.0,-1.0,0.350,0.0,1.0,...,0.0,0.0,2.289,0.550,2.162,0.0,0.0,1.0,1.0,1.0
521,SOM45,08032017_SOM45,532.0,35.0,23.0,4.0,1.0,0.578,0.0,1.0,...,0.0,1.0,2.162,0.350,2.700,0.0,0.0,1.0,1.0,0.0
522,SOM45,08032017_SOM45,533.0,36.0,24.0,5.0,2.0,0.548,0.0,1.0,...,0.0,1.0,2.700,0.578,2.885,0.0,0.0,1.0,1.0,0.0
523,SOM45,08032017_SOM45,534.0,37.0,25.0,6.0,3.0,0.385,0.0,1.0,...,0.0,1.0,2.885,0.548,2.176,0.0,0.0,1.0,0.0,0.0
524,SOM45,08032017_SOM45,535.0,38.0,25.0,7.0,-1.0,0.664,0.0,1.0,...,0.0,0.0,2.176,0.385,19.284,1.0,1.0,0.0,0.0,0.0
525,SOM45,08032017_SOM45,536.0,39.0,25.0,1.0,-2.0,0.481,0.0,1.0,...,1.0,0.0,19.284,0.664,2.021,0.0,1.0,1.0,0.0,0.0


## Save combined feature matrix  

In [28]:
master_matrix.to_csv(os.path.join(root_dir,'0.1opto_trials_9010_high.csv'))